# 📊 Weight Tracking and Prediction

This script visualizes body weight changes from data exported from Apple health app and provides a predictive trend line when a target weight (e.g., 58kg) will be achieved.

Install the Required packages and import the parser.

In [ ]:
!uv add apple_health_parser plotly.express scipy

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from apple_health_parser.utils.parser import Parser
from IPython.display import Image, display
from scipy.optimize import curve_fit
from scipy.optimize import fsolve


Then load health data from zip file, which exported from the Apple health app.

Please note that for data exported from a Chinese system, you need to rename the file inside the ZIP archive from “导出.xml” to “export.xml”, then re-compress the files into a ZIP archive before it can be parsed correctly.

In [ ]:
file = "/Users/aiden/Downloads/apple_health_export.zip"

In [ ]:
# load data
parser = Parser(export_file=file, overwrite=True)
data = parser.get_flag_records(flag="HKQuantityTypeIdentifierBodyMass")

Setup some variables to customize the plot attribute.

In [ ]:
# 🧑 Author and timestamp info
author_name = "Jin"
created_at = pd.Timestamp.now(tz='Asia/Shanghai').isoformat(timespec='seconds')
author_text = f"📊 Created by {author_name}<br>🕒 {created_at}"

# 📝 Text labels
title_text = "📈 2025 Weight Trend"
value_label = "Weight (kg)"
hover_template = "Date: %{x|%Y-%m-%d}<br>Weight: %{y:.1f}kg<extra></extra>"
info_box_template = (
    "📉 Total Loss: {diff}kg<br>"
    "Start ({start_date}): {start:.1f} kg<br>"
    "Current ({end_date}): {end:.1f} kg"
)
max_text = "Max: {:.1f} kg"
min_text = "Min: {:.1f} kg"
avg_line_label = "Avg: {:.1f} kg"
prediction_label = "🎯 Expected {date}<br>to reach {target:.1f} kg"

Create the plot.

In [ ]:
df = data.records.copy()
df['creation_date'] = pd.to_datetime(df['creation_date'])
df = df.sort_values('creation_date')
df = df[df['creation_date'] >= pd.Timestamp('2025-01-01', tz='Asia/Shanghai')]


# Display labels every 5 points
df['text_label'] = df['value'].where(df.index % 8 == 0)
df['text_label'] = df['text_label'].apply(lambda x: f"{x:.1f}" if pd.notnull(x) else '')

# 📊 Key statistics
max_row = df.loc[df['value'].idxmax()]
min_value = df['value'].min()
min_rows = df[df['value'] == min_value]
min_row = min_rows.iloc[-1]
start_row = df.iloc[0]
end_row = df.iloc[-1]
weight_diff = round(start_row['value'] - end_row['value'], 2)
avg_weight = df['value'].mean()

# ℹ️ Summary info box
info_text = info_box_template.format(
    diff=weight_diff,
    start=start_row['value'],
    end=end_row['value'],
    start_date=start_row['creation_date'].date(),
    end_date=end_row['creation_date'].date()
)

# 📈 Main chart
fig = px.line(
    df,
    x='creation_date',
    y='value',
    title=title_text,
    labels={'value': value_label},
    markers=True,
    text='text_label',
    height=600,
    template='plotly_dark'
)

# ✨ Gradient fill under the line
fig.add_trace(go.Scatter(
    x=df['creation_date'],
    y=df['value'],
    fill='tozeroy',
    fillcolor='rgba(0, 230, 118, 0.2)',
    mode='none',
    showlegend=False,
    hoverinfo='none'
))

# ✨ Main curve styling
fig.update_traces(
    selector=dict(type='scatter', mode='lines+markers'),
    texttemplate='%{text}',
    textposition='top center',
    textfont=dict(color='#00e676', size=12),
    hovertemplate=hover_template,
    line=dict(width=3.5, color='#00e676'),
    marker=dict(size=8, color='#00e676', line=dict(color='white', width=2))
)

# 🔺 Max weight label
fig.add_trace(go.Scatter(
    x=[max_row['creation_date']],
    y=[max_row['value']],
    mode='markers+text',
    marker=dict(color='#ff5252', size=12),
    text=[max_text.format(max_row['value'])],
    textposition="top right",
    textfont=dict(color='#ff5252', size=14),
    showlegend=False,
    hoverinfo='none'
))

# 🔻 Min weight label
fig.add_trace(go.Scatter(
    x=[min_row['creation_date']],
    y=[min_row['value']],
    mode='markers+text',
    marker=dict(color='#40c4ff', size=12),
    text=[min_text.format(min_row['value'])],
    textposition="bottom center",
    textfont=dict(color='#40c4ff', size=14),
    showlegend=False,
    hoverinfo='none'
))

# 📏 Average weight horizontal line
fig.add_hline(
    y=avg_weight,
    line_dash="dash",
    line_color="#ffeb3b",
    line_width=2,
    annotation_text=avg_line_label.format(avg_weight),
    annotation_position="top right",
    annotation_font=dict(size=12, color="#ffeb3b")
)

# 📌 Summary info box (top right)
fig.add_annotation(
    text=info_text,
    xref="paper", yref="paper",
    x=1, y=0.99,
    xanchor="right", yanchor="top",
    showarrow=False,
    align="left",
    borderpad=10,
    bgcolor="rgba(30, 30, 30, 0.6)",
    bordercolor="rgba(100, 100, 100, 0.5)",
    borderwidth=1,
    font=dict(size=12, color="#e0e0e0")
)

# 🖊 Author info box (bottom right)
fig.add_annotation(
    text=author_text,
    xref="paper", yref="paper",
    x=1, y=0.01,  # Correct y-coordinate for bottom-right
    xanchor="right", yanchor="bottom",
    showarrow=False,
    align="left",
    font=dict(size=10, color="#a0a0a0", family="Arial"),
    bgcolor="rgba(30, 30, 30, 0.6)",
    bordercolor="rgba(100, 100, 100, 0.5)",
    borderwidth=1,
    borderpad=6
)

# 🎨 Final layout styling
fig.update_layout(
    xaxis=dict(
        tickformat="%Y-%m-%d",
        title=dict(text="", standoff=25, font=dict(size=14)),
        gridcolor='rgba(255, 255, 255, 0.1)'
    ),
    yaxis=dict(
        gridcolor='rgba(255, 255, 255, 0.1)',
        range=[min_row['value'] - 6, max_row['value'] + 2] 
    ),
    margin=dict(l=60, r=40, t=100, b=60),
    title_font=dict(size=24),
    showlegend=False
)

fig.show()
# img_bytes = pio.to_image(fig, format="png", width=1200, height=600, scale=2)
# display(Image(data=img_bytes))

In [ ]:
# 🎯 Target weight
target_weight = 60.0

# 指数衰减函数
def exp_decay(x, a, b, c):
    return a * np.exp(-b * x) + c

# 时间归一化（以天为单位）
df['days'] = (df['creation_date'] - df['creation_date'].min()).dt.days
x = df['days'].values
y = df['value'].values

# 拟合指数衰减
popt, _ = curve_fit(exp_decay, x, y, p0=(y[0]-y[-1], 0.01, y[-1]))

# 预测未来N天
future_days = 65
x_future = np.arange(x[-1], x[-1] + future_days + 1)
y_future = exp_decay(x_future, *popt)
date_future = df['creation_date'].iloc[-1] + pd.to_timedelta(x_future - x[-1], unit='D')

# 预测达到目标体重的天数
def reach_target_func(day):
    return exp_decay(day, *popt) - target_weight

try:
    predicted_day = float(fsolve(reach_target_func, x[-1]+1)[0])
    predicted_date = df['creation_date'].min() + pd.Timedelta(days=predicted_day)
    show_pred = True
except Exception:
    predicted_date = None
    show_pred = False

# 只添加未来的预测曲线
fig.add_trace(go.Scatter(
    x=date_future,
    y=y_future,
    mode='lines',
    name='Prediction',
    line=dict(color='#ff9800', width=2, dash='dot'),
    showlegend=False,
    hovertemplate="Predicted weight: %{y:.1f}kg<br>Date: %{x|%Y-%m-%d}<extra></extra>",
))

# 添加预测目标点的 annotation
if show_pred and predicted_day > x[-1]:
    fig.add_trace(go.Scatter(
        x=[predicted_date],
        y=[target_weight],
        mode='markers',
        marker=dict(size=6, color='#ff9800'),
        showlegend=False
    ))
    fig.add_annotation(
        text=prediction_label.format(date=predicted_date.date(), target=target_weight),
        x=predicted_date,
        y=target_weight,
        showarrow=True,
        arrowcolor="#ff9800",
        arrowhead=1,
        arrowwidth=1,
        ax=0,
        ay=-45,
        bgcolor="rgba(255, 152, 0, 0.2)",
        borderpad=4,
        font=dict(color="#ff9800", size=13)
    )
fig.show()